# Video Subtitle Remover (VSR) - Google Colab

Ứng dụng AI để xóa phụ đề cứng (hardcoded subtitles) khỏi video.

**Tính năng:**
- Xóa phụ đề cứng khỏi video với độ phân giải không mất mát
- Sử dụng AI để điền vào vùng đã xóa phụ đề
- Hỗ trợ tự động phát hiện hoặc chỉ định vùng phụ đề
- Hỗ trợ xử lý ảnh và video

**Thuật toán hỗ trợ:**
- **STTN**: Tốt cho video người thật, nhanh
- **LAMA**: Tốt cho ảnh và video hoạt hình
- **PROPAINTER**: Tốt cho video chuyển động mạnh, cần nhiều VRAM


## 1. Cài đặt môi trường và phụ thuộc


In [ ]:
# Cài đặt các thư viện cần thiết
# Lưu ý: Sử dụng numpy 1.x và protobuf 5.x để tương thích với Colab và các dependencies

print("📦 Bước 1: Cài đặt numpy và protobuf (tương thích với Colab)...")
# Cài đặt numpy và protobuf trước (tương thích với Colab)
!pip install -q "numpy<2.0,>=1.24.4" "protobuf<6.0.0,>=5.26.1"

print("📦 Bước 2: Cài đặt các thư viện chính...")
# Cài đặt các thư viện khác
!pip install -q albumentations==1.4.10 filesplit==3.0.2 opencv-python==4.11.0.86 scikit-image==0.25.2 imgaug==0.4.0
!pip install -q pyclipper==1.3.0.post6 lmdb==1.6.2 PyYAML==6.0.2 omegaconf==2.3.0 tqdm==4.67.1
!pip install -q easydict==1.13 scikit-learn==1.6.1 "pandas==2.2.2" webdataset==0.2.111
!pip install -q av==14.3.0 einops==0.8.1 paddleocr==2.10.0 paddle2onnx==1.3.1
!pip install -q onnxruntime-gpu==1.20.1

print("📦 Bước 3: Cài đặt PyTorch với CUDA (với CUDA libraries mới)...")
# Cài đặt PyTorch với CUDA (Colab có GPU)
# PyTorch 2.7.0 yêu cầu nvidia-cudnn-cu11==9.1.0.70 và nvidia-nccl-cu11==2.21.5
!pip install -q torch==2.7.0 torchvision==0.22.0 --index-url https://download.pytorch.org/whl/cu118

print("📦 Bước 4: Cài đặt PaddlePaddle GPU (sẽ có warning về CUDA libraries)...")
# Cài đặt PaddlePaddle GPU
# PaddlePaddle yêu cầu CUDA libraries cũ hơn, nhưng sẽ hoạt động với libraries mới hơn
!pip install -q paddlepaddle-gpu==3.0.0 -i https://www.paddlepaddle.org.cn/packages/stable/cu118/ --no-deps || \
!pip install -q paddlepaddle-gpu==3.0.0 -i https://www.paddlepaddle.org.cn/packages/stable/cu118/

print("\n📦 Bước 5: Xử lý xung đột dependencies...")
# Xử lý xung đột với shap và các package khác
!pip install -q --force-reinstall "numpy<2.0,>=1.24.4" "protobuf<6.0.0,>=5.26.1" 2>&1 | grep -v "ERROR" || true

# Xử lý torchaudio conflict (nếu có)
!pip uninstall -y torchaudio 2>/dev/null || true

print("\n✅ Đã cài đặt xong các thư viện!")
print("\n📝 Lưu ý về các warnings:")
print("   ✅ numpy và protobuf: Đã điều chỉnh để tương thích với Colab")
print("   ⚠️  CUDA libraries: PyTorch và PaddlePaddle yêu cầu versions khác nhau")
print("      - PyTorch dùng CUDA libraries mới hơn (9.1.0.70)")
print("      - PaddlePaddle yêu cầu CUDA libraries cũ hơn (8.9.6.50)")
print("      - Điều này là BÌNH THƯỜNG và cả hai vẫn hoạt động được")
print("   ⚠️  torchaudio: Đã gỡ bỏ nếu có conflict (không cần thiết cho dự án này)")
print("   ⚠️  shap, jax, jaxlib: Không ảnh hưởng vì dự án không sử dụng trực tiếp")


In [ ]:
# Kiểm tra versions của các thư viện quan trọng
import sys
print("📦 Kiểm tra versions:")
print("=" * 50)

try:
    import numpy
    print(f"✅ numpy: {numpy.__version__}")
except:
    print("❌ numpy: KHÔNG TÌM THẤY")

try:
    import protobuf
    print(f"✅ protobuf: {protobuf.__version__}")
except:
    try:
        import google.protobuf
        print(f"✅ protobuf: {google.protobuf.__version__}")
    except:
        print("❌ protobuf: KHÔNG TÌM THẤY")

try:
    import pandas
    print(f"✅ pandas: {pandas.__version__}")
except:
    print("❌ pandas: KHÔNG TÌM THẤY")

try:
    import torch
    print(f"✅ torch: {torch.__version__}")
    print(f"✅ CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
        print(f"✅ CUDA version: {torch.version.cuda}")
        print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    else:
        print("⚠️ Không có GPU, quá trình xử lý sẽ chậm hơn!")
except:
    print("❌ torch: KHÔNG TÌM THẤY")

try:
    import cv2
    print(f"✅ opencv-python: {cv2.__version__}")
except:
    print("❌ opencv-python: KHÔNG TÌM THẤY")

print("=" * 50)


### 🔧 Xử lý xung đột dependencies

**Lưu ý về các warnings:**

1. **CUDA Libraries Conflict** (nvidia-cudnn-cu11, nvidia-nccl-cu11):
   - PyTorch và PaddlePaddle yêu cầu versions khác nhau
   - **Giải pháp:** Giữ libraries mới hơn (theo PyTorch)
   - **Kết quả:** Cả hai vẫn hoạt động bình thường (backward compatible)
   - ✅ **Bình thường** và **không ảnh hưởng**

2. **numpy/shap/jax conflicts:**
   - Các warnings về `shap`, `jax`, `jaxlib`, `pytensor` yêu cầu `numpy>=2.0` là **bình thường**
   - Dự án **không sử dụng** các package này trực tiếp
   - Các dependencies chính yêu cầu `numpy<2.0`
   - ✅ **Bình thường** và **không ảnh hưởng**

3. **torchaudio conflict:**
   - Đã tự động gỡ bỏ vì không cần thiết cho dự án

**Kết luận:** Tất cả các warnings này có thể **BỎ QUA** và tiếp tục sử dụng!


In [ ]:
# Fix dependencies conflicts
# Xử lý các xung đột với shap, jax, và các package khác yêu cầu numpy>=2

print("🔧 Đang xử lý xung đột dependencies...")

# Downgrade các package yêu cầu numpy>=2 về version tương thích với numpy 1.x
# Shap không phải dependency chính của dự án này, nên có thể downgrade
try:
    !pip install -q --force-reinstall "shap<0.50.0" 2>/dev/null || echo "⚠️ Không thể downgrade shap"
except:
    pass

# Fix numpy và protobuf nếu cần
!pip install -q --force-reinstall "numpy<2.0,>=1.24.4" "protobuf<6.0.0,>=5.26.1"

print("✅ Đã xử lý xung đột dependencies!")
print("\n💡 Lưu ý:")
print("   - Các warning về shap, jax, jaxlib có thể xuất hiện nhưng không ảnh hưởng đến chức năng chính")
print("   - Dự án này không sử dụng shap, jax, jaxlib trực tiếp")
print("   - Nếu vẫn gặp lỗi, hãy restart runtime và chạy lại từ đầu")


### ⚠️ Giải thích về xung đột CUDA Libraries

**Vấn đề:**
- `torch 2.7.0+cu118` yêu cầu: `nvidia-cudnn-cu11==9.1.0.70`, `nvidia-nccl-cu11==2.21.5`
- `paddlepaddle-gpu 3.0.0` yêu cầu: `nvidia-cudnn-cu11==8.9.6.50`, `nvidia-nccl-cu11==2.19.3`

**Giải pháp:**
- Giữ CUDA libraries mới hơn (theo PyTorch) vì PyTorch quan trọng hơn cho dự án
- PaddlePaddle vẫn hoạt động được với CUDA libraries mới hơn (backward compatible)
- Các warnings này là **BÌNH THƯỜNG** và **KHÔNG ẢNH HƯỞNG** đến chức năng

**Kết luận:** Bạn có thể bỏ qua các warnings này và tiếp tục sử dụng!


## 2. Tải xuống mã nguồn dự án


In [ ]:
# Clone repository hoặc tải từ GitHub
import os
import shutil

# Xóa thư mục cũ nếu có
if os.path.exists('video-subtitle-remover'):
    shutil.rmtree('video-subtitle-remover')

# Clone repository
!git clone -q https://github.com/YaoFANGUK/video-subtitle-remover.git

# Di chuyển vào thư mục dự án
os.chdir('video-subtitle-remover')

print("✅ Đã tải xuống mã nguồn!")


## 3. Cài đặt FFmpeg


In [ ]:
# Cài đặt FFmpeg cho Linux (Colab chạy trên Linux)
!apt-get update -qq
!apt-get install -y -qq ffmpeg

# Kiểm tra FFmpeg
!ffmpeg -version | head -n 1

print("✅ FFmpeg đã sẵn sàng!")


## 4. Tải xuống và thiết lập mô hình

**Lưu ý:** Các mô hình có thể rất lớn (vài GB). Bạn cần tải chúng từ GitHub releases hoặc upload từ máy tính.


In [ ]:
import os
from pathlib import Path

# Tạo thư mục models nếu chưa có
os.makedirs('backend/models', exist_ok=True)
os.makedirs('backend/models/big-lama', exist_ok=True)
os.makedirs('backend/models/sttn', exist_ok=True)
os.makedirs('backend/models/video', exist_ok=True)
os.makedirs('backend/models/V4/ch_det', exist_ok=True)

print("📁 Đã tạo cấu trúc thư mục models")
print("\n⚠️ LƯU Ý: Bạn cần tải các file mô hình từ:")
print("   - GitHub Releases: https://github.com/YaoFANGUK/video-subtitle-remover/releases")
print("   - Hoặc sử dụng Google Drive để upload các file mô hình đã tải về.")


In [ ]:
# Upload models từ máy tính hoặc Google Drive
from google.colab import drive, files

# Mount Google Drive (nếu bạn đã lưu models trên Drive)
# drive.mount('/content/drive')

print("📤 Nếu bạn có file models, hãy upload chúng:")
print("   1. big-lama.pt -> backend/models/big-lama/")
print("   2. infer_model.pth -> backend/models/sttn/")
print("   3. ProPainter.pth -> backend/models/video/")
print("   4. raft-things.pth -> backend/models/video/")
print("   5. recurrent_flow_completion.pth -> backend/models/video/")
print("   6. inference.pdmodel, inference.pdiparams -> backend/models/V4/ch_det/")

# Uncomment dòng dưới để upload files
# uploaded = files.upload()


In [ ]:
# Script kiểm tra models
import os

models_to_check = {
    'big-lama.pt': 'backend/models/big-lama/big-lama.pt',
    'infer_model.pth': 'backend/models/sttn/infer_model.pth',
    'ProPainter.pth': 'backend/models/video/ProPainter.pth',
    'raft-things.pth': 'backend/models/video/raft-things.pth',
    'recurrent_flow_completion.pth': 'backend/models/video/recurrent_flow_completion.pth',
    'inference.pdmodel': 'backend/models/V4/ch_det/inference.pdmodel',
    'inference.pdiparams': 'backend/models/V4/ch_det/inference.pdiparams',
}

print("📦 Kiểm tra models:")
print("=" * 50)
for name, path in models_to_check.items():
    exists = os.path.exists(path)
    status = "✅" if exists else "❌"
    if exists:
        size = os.path.getsize(path) / (1024**2)  # MB
        print(f"{status} {name:30s} ({size:.2f} MB)")
    else:
        print(f"{status} {name:30s} (THIẾU)")

print("\n💡 Nếu thiếu models, hãy tải từ GitHub releases và upload vào Colab")


## 5. Cấu hình (Tùy chọn)

Bạn có thể chỉnh sửa file `backend/config.py` để thay đổi:
- Thuật toán (STTN, LAMA, PROPAINTER)
- Các tham số xử lý
- Vùng phụ đề


In [ ]:
# Xem cấu hình hiện tại
import sys
sys.path.insert(0, 'backend')

try:
    import config
    print(f"MODE: {config.MODE}")
    print(f"STTN_SKIP_DETECTION: {config.STTN_SKIP_DETECTION}")
    print(f"STTN_NEIGHBOR_STRIDE: {config.STTN_NEIGHBOR_STRIDE}")
    print(f"STTN_REFERENCE_LENGTH: {config.STTN_REFERENCE_LENGTH}")
    print(f"STTN_MAX_LOAD_NUM: {config.STTN_MAX_LOAD_NUM}")
    print(f"\nDevice: {config.device}")
    print(f"USE_DML: {config.USE_DML}")
except Exception as e:
    print(f"Lỗi khi đọc config: {e}")


## 6. Upload video/ảnh cần xử lý


In [ ]:
from google.colab import files
import os

# Tạo thư mục input
os.makedirs('input', exist_ok=True)

print("📤 Hãy upload video hoặc ảnh cần xử lý:")
uploaded = files.upload()

# Di chuyển file vào thư mục input
for filename in uploaded.keys():
    os.rename(filename, f'input/{filename}')
    print(f"✅ Đã upload: {filename}")


## 7. Xử lý video/ảnh

Chạy script để xóa phụ đề. Bạn có thể:
- Để `sub_area=None` để tự động phát hiện phụ đề
- Hoặc chỉ định vùng phụ đề: `sub_area=(ymin, ymax, xmin, xmax)`


In [ ]:
import sys
import os
import multiprocessing

# Thêm đường dẫn vào sys.path
sys.path.insert(0, os.getcwd())
sys.path.insert(0, 'backend')

# Thiết lập multiprocessing
multiprocessing.set_start_method("spawn", force=True)

# Import các module cần thiết
from backend.main import SubtitleRemover
from backend.tools.common_tools import is_video_or_image

# Đường dẫn video/ảnh
input_dir = 'input'
input_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

if not input_files:
    print("❌ Không tìm thấy file nào trong thư mục input!")
    print("   Hãy upload file ở bước 6.")
else:
    # Xử lý file đầu tiên (hoặc bạn có thể chỉ định file cụ thể)
    video_path = os.path.join(input_dir, input_files[0])
    
    print(f"🎬 Bắt đầu xử lý: {video_path}")
    print(f"📊 File size: {os.path.getsize(video_path) / (1024*1024):.2f} MB")
    
    # Kiểm tra file hợp lệ
    if is_video_or_image(video_path):
        # sub_area=None: Tự động phát hiện phụ đề
        # sub_area=(ymin, ymax, xmin, xmax): Chỉ định vùng phụ đề
        # Ví dụ: sub_area=(400, 500, 100, 900) - vùng phụ đề ở dưới cùng
        sub_area = None  # Thay đổi nếu muốn chỉ định vùng
        
        # Tạo đối tượng SubtitleRemover
        remover = SubtitleRemover(video_path, sub_area=sub_area, gui_mode=False)
        
        # Chạy xử lý
        print("\n🚀 Bắt đầu xóa phụ đề...")
        remover.run()
        
        print("\n✅ Hoàn thành!")
        print(f"📁 File output: {remover.video_out_name}")
    else:
        print(f"❌ File không hợp lệ: {video_path}")


## 8. Tải xuống kết quả


In [ ]:
from google.colab import files
import os

# Tìm file output
output_file = None

# Tìm trong thư mục input (file output thường ở cùng thư mục với input)
input_dir = 'input'
if os.path.exists(input_dir):
    for f in os.listdir(input_dir):
        if '_no_sub' in f or 'no_sub' in f:
            output_file = os.path.join(input_dir, f)
            break

# Nếu không tìm thấy, tìm trong thư mục gốc
if not output_file:
    for root, dirs, files_list in os.walk('.'):
        for f in files_list:
            if '_no_sub' in f or (f.endswith('.mp4') and 'no_sub' in root):
                output_file = os.path.join(root, f)
                break
        if output_file:
            break

if output_file and os.path.exists(output_file):
    print(f"📥 Tải xuống file: {output_file}")
    files.download(output_file)
else:
    print("❌ Không tìm thấy file output!")
    print("\n📁 Tìm kiếm file output...")
    for root, dirs, files_list in os.walk('.'):
        for f in files_list:
            if '_no_sub' in f or f.endswith(('.mp4', '.png', '.jpg')):
                print(f"   Tìm thấy: {os.path.join(root, f)}")


## 9. Xử lý hàng loạt (Tùy chọn)

Nếu bạn có nhiều video/ảnh cần xử lý:


In [ ]:
import sys
import os
import multiprocessing

sys.path.insert(0, os.getcwd())
sys.path.insert(0, 'backend')

multiprocessing.set_start_method("spawn", force=True)

from backend.main import SubtitleRemover
from backend.tools.common_tools import is_video_or_image

input_dir = 'input'
input_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

if not input_files:
    print("❌ Không có file nào để xử lý!")
else:
    print(f"📋 Tìm thấy {len(input_files)} file(s)")
    
    # Xử lý từng file
    for idx, filename in enumerate(input_files, 1):
        video_path = os.path.join(input_dir, filename)
        
        if not is_video_or_image(video_path):
            print(f"⏭️  Bỏ qua file không hợp lệ: {filename}")
            continue
        
        print(f"\n{'='*60}")
        print(f"📹 [{idx}/{len(input_files)}] Xử lý: {filename}")
        print(f"{'='*60}")
        
        try:
            # Tự động phát hiện phụ đề
            sub_area = None
            
            remover = SubtitleRemover(video_path, sub_area=sub_area, gui_mode=False)
            remover.run()
            
            print(f"✅ Hoàn thành: {remover.video_out_name}")
        except Exception as e:
            print(f"❌ Lỗi khi xử lý {filename}: {e}")
            import traceback
            traceback.print_exc()
    
    print(f"\n🎉 Đã xử lý xong tất cả {len(input_files)} file(s)!")


## 💡 Mẹo và Lưu ý

1. **Chọn thuật toán phù hợp:**
   - **STTN**: Nhanh, tốt cho video người thật
   - **LAMA**: Tốt cho ảnh và video hoạt hình
   - **PROPAINTER**: Tốt nhất nhưng cần nhiều VRAM

2. **Tối ưu tốc độ:**
   - Bật `STTN_SKIP_DETECTION = True` để bỏ qua phát hiện phụ đề
   - Giảm `STTN_MAX_LOAD_NUM` nếu thiếu VRAM

3. **Chất lượng:**
   - Tăng `STTN_REFERENCE_LENGTH` và `STTN_NEIGHBOR_STRIDE` để cải thiện chất lượng
   - Nhưng sẽ chậm hơn và cần nhiều VRAM hơn

4. **Vùng phụ đề:**
   - Để `sub_area=None` để tự động phát hiện
   - Hoặc chỉ định: `sub_area=(ymin, ymax, xmin, xmax)`
   - Ví dụ: `sub_area=(400, 500, 100, 900)` - vùng ở dưới cùng

5. **VRAM:**
   - Colab free: ~15GB VRAM
   - Colab Pro: ~25GB VRAM
   - Điều chỉnh `STTN_MAX_LOAD_NUM` và `PROPAINTER_MAX_LOAD_NUM` theo VRAM


## 🔧 Xử lý sự cố

### Lỗi thiếu models:
- Kiểm tra models đã có chưa bằng cell ở trên
- Tải models từ GitHub releases và upload vào Colab

### Lỗi CUDA out of memory:
- Giảm `STTN_MAX_LOAD_NUM` trong `backend/config.py`
- Hoặc sử dụng LAMA mode thay vì STTN

### Lỗi import:
- Chạy lại cell cài đặt dependencies
- Hoặc restart runtime và chạy lại từ đầu
